To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth ahora es compatible con modelos de Texto a Voz (TTS). Lee nuestra guía aquí.

Lee nuestra **Guía de Qwen3** y echa un vistazo a nuestras nuevas cuantizaciones **Dynamic 2.0** que superan a otros métodos de cuantización!
Visita nuestra documentación para todas nuestras subidas de modelos y notebooks.

### Instalacion de dependencias

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Ahora agregamos adaptadores LoRA para que solo necesitemos actualizar del 1 al 10% de todos los parámetros!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
Preparación de Datos

Ahora usamos el formato `Phi-3` para ajustes finos de estilo conversacional. Utilizamos conversaciones de Open Assistant en estilo ShareGPT. Phi-3 renderiza conversaciones multi-turno como se muestra a continuación:

```
<|user|>
¡Hola!<|end|>
<|assistant|>
¡Hola! ¿Cómo estás?<|end|>
<|user|>
¡Estoy muy bien! ¿Y tú?<|end|>

```

[NOTA] Para entrenar solo en las completaciones (ignorando la entrada del usuario) lee la documentación de Unsloth aquí.
Usamos nuestra función get_chat_template para obtener la plantilla de chat correcta. Soportamos zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old y nuestra propia plantilla optimizada unsloth.
Nota que ShareGPT usa {"from": "human", "value" : "Hola"} y no {"role": "user", "content" : "Hola"}, por eso usamos mapping para mapearlo.
Para completaciones de texto como escritura de novelas, prueba este notebook.
Note ShareGPT uses `{"from": "human", "value" : "Hi"}` and not `{"role": "user", "content" : "Hi"}`, so we use `mapping` to map it.

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [5]:
from unsloth.chat_templates import get_chat_template
# Necesitarás importar AutoTokenizer si aún no lo has hecho en una celda anterior
# from transformers import AutoTokenizer

# Asumiendo que 'tokenizer' ya está definida en una celda anterior o que la definirás aquí:
# Si el tokenizer base NO ha sido cargado todavía, DESCOMENTA la siguiente línea
# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-small-8k-instruct")

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# Elimina 'pass' si está en tu código, ya que no es necesario aquí.

from datasets import load_dataset

# *** CAMBIO PRINCIPAL AQUÍ ***
# Cargar tu archivo Administracion.json
# Asegúrate de que Administracion.json esté en /content/ si lo subiste directamente
dataset = load_dataset("json", data_files="/content/Administracion.json", split = "train")

# Aplicar la función de formateo a tu dataset recién cargado
dataset = dataset.map(formatting_prompts_func, batched = True,)

# Opcional: Imprimir un elemento para verificar que todo se cargó y formateó correctamente
print("\n--- Verificación del formato después de aplicar el chat_template ---")
if len(dataset) > 0:
    print(dataset[0]["text"])
else:
    print("El dataset está vacío o no se cargó correctamente.")
print("\n--- Dataset 'Administracion.json' cargado y formateado correctamente para entrenamiento. ---")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]


--- Verificación del formato después de aplicar el chat_template ---
<|user|>
¿Qué es la Licenciatura en Administración de Empresas de la UFG?<|end|>
<|assistant|>
La Licenciatura en Administración de Empresas en Modalidad Semipresencial de la UFG ofrece la preparación y formación de profesionales usando Educación a Distancia y Presencial con herramientas de plataforma virtual. Está diseñada para proporcionar a la Sociedad Salvadoreña el talento humano calificado necesario para el desarrollo socio-económico, siendo una oportunidad para bachilleres, técnicos o profesionales con dificultades de tiempo y movilización para asistir a clases presenciales.<|end|>


--- Dataset 'Administracion.json' cargado y formateado correctamente para entrenamiento. ---


Veamos cómo funciona el formato Phi-3 imprimiendo el 5º elemento

In [ ]:
dataset[5]["conversations"]

[{'from': 'human',
  'value': 'What is the typical wattage of bulb in a lightbox?'},
 {'from': 'gpt',
  'value': 'The typical wattage of a bulb in a lightbox is 60 watts, although domestic LED bulbs are normally much lower than 60 watts, as they produce the same or greater lumens for less wattage than alternatives. A 60-watt Equivalent LED bulb can be calculated using the 7:1 ratio, which divides 60 watts by 7 to get roughly 9 watts.'},
 {'from': 'human',
  'value': 'Rewrite your description of the typical wattage of a bulb in a lightbox to only include the key points in a list format.'}]

In [ ]:
print(dataset[5]["text"])

<|user|>
What is the typical wattage of bulb in a lightbox?<|end|>
<|assistant|>
The typical wattage of a bulb in a lightbox is 60 watts, although domestic LED bulbs are normally much lower than 60 watts, as they produce the same or greater lumens for less wattage than alternatives. A 60-watt Equivalent LED bulb can be calculated using the 7:1 ratio, which divides 60 watts by 7 to get roughly 9 watts.<|end|>
<|user|>
Rewrite your description of the typical wattage of a bulb in a lightbox to only include the key points in a list format.<|end|>



¡Si buscas crear tu propia plantilla de chat, eso también es posible! Debes usar el régimen de plantillas Jinja. Proporcionamos nuestra propia versión simplificada de la `plantilla Unsloth` que encontramos más eficiente, y aprovecha los estilos ChatML, Zephyr y Alpaca.

¡Más información sobre plantillas de chat en nuestra página wiki!

In [8]:
# Definición del template personalizado para el chat
# Este template define cómo se formatean los mensajes en el modelo
unsloth_template = ( # Abre paréntesis aquí
    "{{ bos_token }}" # Cada segmento de la cadena va en su propia línea, sin \ al final
    "{{ 'Eres un asistente virtual de la Universidad Francisco Gavidia\n' }}"
    "{% for message in messages %}"
        "{% if message['role'] == 'user' %}"
            "{{ '>>> User: ' + message['content'] + '\n' }}"
        "{% elif message['role'] == 'assistant' %}"
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"
        "{% endif %}"
    "{% endfor %}"
    "{% if add_generation_prompt %}"
        "{{ '>>> Assistant: ' }}"
    "{% endif %}"
) # Cierra paréntesis aquí. No se necesita \ en ninguna parte.

# Define el token de fin de secuencia personalizado
unsloth_eos_token = "eos_token"

# Código condicional (actualmente deshabilitado con if False)
if False: # Recuerda cambiar esto a 'if True:' cuando quieras aplicar este template
    # Función para aplicar el template de chat personalizado al tokenizador
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,),
        mapping = {
            "role" : "from",
            "content" : "value",
            "user" : "human",
            "assistant" : "gpt"
        },
        map_eos_token = True,
    )

<a name="Train"></a>
### Entrenar el modelo
¡Ahora usemos el `SFTTrainer` de Hugging Face TRL! Más documentación aquí: [documentos TRL SFT](https://huggingface.co/docs/trl/sft_trainer). Hacemos 60 pasos para acelerar las cosas, pero puedes establecer `num_train_epochs=1` para una ejecución completa, y desactivar `max_steps=None`. ¡También soportamos el `DPOTrainer` de TRL!

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.648 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20 | Num Epochs = 20 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416/2,039,024,640 (1.47% trained)


Step,Training Loss
1,2.154100
2,2.376900
3,2.256100
4,2.241400
5,2.048300
6,1.751800
7,1.764300
8,1.692800
9,1.438800
10,1.373600


In [12]:
# @title Mostrar las estadísticas finales de memoria y tiempo
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} segundos usados para el entrenamiento.")
print(
   f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutos usados para el entrenamiento."
)
print(f"Memoria máxima reservada = {used_memory} GB.")
print(f"Memoria máxima reservada para entrenamiento = {used_memory_for_lora} GB.")
print(f"Memoria máxima reservada % de memoria máxima = {used_percentage} %.")
print(f"Memoria máxima reservada para entrenamiento % de memoria máxima = {lora_percentage} %.")

132.0188 segundos usados para el entrenamiento.
2.2 minutos usados para el entrenamiento.
Memoria máxima reservada = 3.648 GB.
Memoria máxima reservada para entrenamiento = 0.0 GB.
Memoria máxima reservada % de memoria máxima = 24.747 %.
Memoria máxima reservada para entrenamiento % de memoria máxima = 0.0 %.


<a name="Inference"></a>
### Inferencia
¡Ejecutemos el modelo! Como estamos usando `Phi-3`, usa `apply_chat_template` con `add_generation_prompt` establecido en `True` para la inferencia.

In [13]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|user|> Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|end|><|assistant|> The next number in the Fibonopsis sequence would be 13 (8 + 5). The sequence would continue as 21 (13 + 8), 34 (21 + 13), and so on.<|end|><|endoftext|>']

#  También puedes usar un `TextStreamer` para inferencia continua - ¡así puedes ver la generación token por token, en lugar de esperar todo el tiempo!

In [14]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

The next number in the Fibonopsis sequence would be 13 (8 + 5). The sequence would continue as 21 (13 + 8), 34 (21 + 13), and so on.<|end|><|endoftext|>


Guardar y cargar modelos ajustados finamente
Para guardar el modelo final como adaptadores LoRA, usa push_to_hub de Hugging Face para un guardado en línea o save_pretrained para un guardado local.
[NOTA] Esto SOLO guarda los adaptadores LoRA, y no el modelo completo. ¡Para guardar en 16bit o GGUF, desplázate hacia abajo!

In [15]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Ahora si quieres cargar los adaptadores LoRA que acabamos de guardar para inferencia, cambia `False` a `True`:

In [16]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "What is a famous tall tower in Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

The Eiffel Tower, located in Paris, is a famous tall tower known for its iron lattice structure and panoramic views of the city. It is one of the most recognizable landmarks in the world.<|end|><|endoftext|>


También puedes usar `AutoModelForPeftCausalLM` de Hugging Face. Solo usa esto si no tienes `unsloth` instalado. Puede ser terriblemente lento, ya que la descarga de modelos `4bit` no está soportada, y **la inferencia de Unsloth es 2x más rápida**.

In [17]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer

    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Guardar en float16 para VLLM

También soportamos guardar directamente en `float16`. Selecciona `merged_16bit` para float16 o `merged_4bit` para int4. También permitimos adaptadores `lora` como respaldo. ¡Usa `push_to_hub_merged` para subir a tu cuenta de Hugging Face! Puedes ir a https://huggingface.co/settings/tokens para obtener tus tokens personales.

In [18]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### Conversión GGUF / llama.cpp
Para guardar en `GGUF` / `llama.cpp`, ¡ahora lo soportamos nativamente! Clonamos `llama.cpp` y por defecto lo guardamos en `q8_0`. Permitimos todos los métodos como `q4_k_m`. Usa `save_pretrained_gguf` para guardado local y `push_to_hub_gguf` para subir a HF.

Algunos métodos de cuantización soportados (lista completa en nuestra [página Wiki](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Conversión rápida. Alto uso de recursos, pero generalmente aceptable.
* `q4_k_m` - Recomendado. Usa Q6_K para la mitad de los tensores attention.wv y feed_forward.w2, el resto Q4_K.
* `q5_k_m` - Recomendado. Usa Q6_K para la mitad de los tensores attention.wv y feed_forward.w2, el resto Q5_K.

[**NUEVO**] Para ajuste fino y exportación automática a Ollama, prueba nuestro [notebook de Ollama](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [21]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

In [22]:
# Asumiendo que 'model' es tu objeto FastLanguageModel de Unsloth
# y 'tokenizer' es tu tokenizador ya cargado y configurado

# *** Paso crucial: Fusionar los adaptadores LoRA con el modelo base ***
# Esto crea un modelo completo y autocontenido en formato GGUF.
model.config.use_cache = True # Es buena práctica asegurarse de que use_cache esté en True

# Opcional: Si tienes problemas de memoria (VRAM)
# del model # Descomenta esto para liberar memoria antes de la fusión, si es necesario

# Definir la carpeta de salida para el modelo GGUF fusionado.
# Es importante que sea una carpeta NUEVA, no 'lora_model', para no sobreescribir tus adaptadores.
output_dir_gguf = "/content/merged_model_for_ollama" # O "/content/drive/MyDrive/tu_modelo_unsloth_gguf"

# Descomenta y ejecuta UNA de estas líneas para guardar el modelo GGUF fusionado.
# La cuantificación "q4_k_m" es un buen balance entre tamaño y calidad para Ollama.
print(f"Guardando el modelo GGUF fusionado en: {output_dir_gguf}...")
model.save_pretrained_gguf(output_dir_gguf, tokenizer, quantization_method = "q4_k_m")

# Opcional: Si quieres otras cuantificaciones:
# model.save_pretrained_gguf(output_dir_gguf, tokenizer, quantization_method = "f16") # Mayor calidad, mayor tamaño
# model.save_pretrained_gguf(output_dir_gguf, tokenizer, quantization_method = "q8_0") # Compromiso

print(f"¡Modelo GGUF fusionado guardado exitosamente en: {output_dir_gguf}!")

Guardando el modelo GGUF fusionado en: /content/merged_model_for_ollama...


Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.3G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.03 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:01<00:00, 18.20it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving /content/merged_model_for_ollama/pytorch_model-00001-of-00002.bin...
Unsloth: Saving /content/merged_model_for_ollama/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at /content/merged_model_for_ollama into f16 GGUF format.
The output location will be /content/merged_model_for_ollama/unsloth.F16.gguf
This might take 3 minutes...


Unsloth: Extending /content/merged_model_for_ollama/tokenizer.model with added_tokens.json.
Originally tokenizer.model is of size (32000).
But we need to extend to sentencepiece vocab size (32011).


INFO:hf-to-gguf:Loading model: merged_model_for_ollama
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00002.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {3072, 32064}
INFO:hf-to-gguf:blk.0.attn_q.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_v.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {3072, 8192}
INFO:

Ahora, usa el archivo `model-unsloth.gguf` o `model-unsloth-Q4_K_M.gguf` en llama.cpp o un sistema basado en UI como Jan u Open WebUI. Puedes instalar Jan [aquí](https://github.com/janhq/jan) y Open WebUI [aquí](https://github.com/open-webui/open-webui)

¡Y hemos terminado! Si tienes alguna pregunta sobre Unsloth, ¡tenemos un canal de [Discord](https://discord.gg/unsloth)! Si encuentras algún error o quieres mantenerte actualizado con las últimas noticias de LLM, o necesitas ayuda, unirte a proyectos, etc., ¡siéntete libre de unirte a nuestro Discord!

Algunos otros enlaces:
1. Entrena tu propio modelo de razonamiento - Notebook Llama GRPO [Colab Gratuito](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Guardar ajustes finos en Ollama. [Notebook gratuito](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Ajuste fino de Llama 3.2 Vision - Caso de uso de radiografía. [Colab Gratuito](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. ¡Ve los notebooks para DPO, ORPO, Pre-entrenamiento continuo, ajuste fino conversacional y más en nuestra [documentación](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Únete a Discord si necesitas ayuda + ⭐️ <i>Danos una estrella en <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>